In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Importing Libraries

In [21]:
!pip install xgboost

In [1]:
from xgboost import XGBRegressor
from sklearn import metrics

In [2]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split

In [3]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
df_train = pd.read_csv('/kaggle/input/amazon-ml-challenge-2023/datasetb2d9982/dataset/train.csv')
#df_test = pd.read_csv('/kaggle/input/amazon-ml-challenge-2023/datasetb2d9982/dataset/test.csv')

In [5]:
df_train.dropna(inplace = True)
df_train.isna().sum()

PRODUCT_ID         0
TITLE              0
BULLET_POINTS      0
DESCRIPTION        0
PRODUCT_TYPE_ID    0
PRODUCT_LENGTH     0
dtype: int64

# Data Preprocessing

In [6]:
def tokenize_text(text):
    # Remove emojis
    text = re.sub(r'[^\w\s]', '', text)
    
    # Remove hyperlinks
    text = re.sub(r'http\S+', '', text)
    
    # Remove hashtags
    text = re.sub(r'#\w+', '', text)
    
    # Convert capital letters to lowercase
    text = text.lower()
    
    # Tokenize the text
    tokens = re.findall(r'\b\w+\b', text)
    
    return tokens

In [14]:
#df_train['TITLE'] = df_train['TITLE'].apply(tokenize_text)
#df_train['BULLET_POINTS'] = df_train['BULLET_POINTS'].apply(tokenize_text)
df_train['DESCRIPTION'] = df_train['DESCRIPTION'].apply(tokenize_text)

In [8]:
def join_list_elements(lst):
    return ' '.join(lst)

In [15]:
#df_train['TITLE'] = df_train['TITLE'].apply(join_list_elements)
#df_train['BULLET_POINTS'] = df_train['BULLET_POINTS'].apply(join_list_elements)
df_train['DESCRIPTION'] = df_train['DESCRIPTION'].apply(join_list_elements)

In [16]:
df_train

,PRODUCT_ID,TITLE,BULLET_POINTS,DESCRIPTION,PRODUCT_TYPE_ID,PRODUCT_LENGTH
2,2765088,priknik horn red electric air horn compressor ...,loud dual tone trumpet horn compatible with sx...,specifications color red material aluminium vo...,7537,748.031495
3,1594019,alishah womens cotton ankle length leggings co...,made by 95cotton and 5 lycra which gives you 4...,aishah womens lycra cotton ankel leggings bran...,2996,787.401574
5,2152929,hins metal bucket shape plant pot for indoor o...,simple and elegant great for displaying indoor...,hins brings you the most elegant looking pot w...,5725,950.000000
7,2026580,delavala self adhesive kitchen backsplash wall...,high quality pvc material the kitchen aluminum...,pstrongaluminum foil stickersgood kitchen help...,6030,984.251967
9,2998633,hexwell essential oil for home fragrance oil a...,100 pure and natural essential oil or fragranc...,transform your home workplace or hotel room in...,8201,393.700787
...,...,...,...,...,...,...
2249686,1898841,cocomii holographic iphone 11 pro case hologra...,cocomii unique stylish quality this beautiful ...,bcocomiibbrbrwe transform amazing ideas and ad...,12556,600.000000
2249689,1616561,have it tall mens curved hem long drop tail t ...,light weight 65 polyester 35 ring spun cotton ...,this extra long tall tshirt will be your favor...,2879,1200.000000
2249694,2766635,3pcs goose game cute cartoon enamel pins funny...,inspiration inspired by the untitled goose gam...,pbbrand bxvieonrp pbrp pbproduct name bfashion...,3413,125.984252
2249695,1987786,kangroo sweep movement printed wooden wall clo...,dial size 12 inches in diameterbig clear repre...,wall clocks are very attractive in looks and e...,1574,1200.000000


In [17]:
df_train['text_data'] = df_train['TITLE'] + ' ' + df_train['DESCRIPTION'] + ' ' + df_train['BULLET_POINTS']

In [18]:
stop_words = set(stopwords.words('english'))
text_data = [word for word in df_train['text_data'] if word.lower() not in stop_words]

In [19]:
df = df_train.drop(['TITLE', 'DESCRIPTION', 'BULLET_POINTS'], axis=1)

In [20]:
tfidf_vectorizer =  TfidfVectorizer()
Vec = tfidf_vectorizer.fit_transform(text_data)

In [21]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(Vec, df['PRODUCT_LENGTH'], test_size=0.2, random_state=42)

In [22]:
# Print the shapes of the resulting datasets
print("Training data shape:", X_train.shape)
print("Testing data shape:", X_test.shape)
print("Training target shape:", y_train.shape)
print("Testing target shape:", y_test.shape)

Training data shape: (830768, 3217513)
Testing data shape: (207692, 3217513)
Training target shape: (830768,)
Testing target shape: (207692,)


# Model Building

In [25]:
xgb2 = XGBRegressor(n_estimators = 1000, learning_rate = 0.001, random_state = 42)

In [ ]:
xgb2.fit(X_train,y_train)

In [ ]:
y_pred = xgb2.predict(y_test)

In [ ]:
rmsle = max(0,100*(1-metrics.mean_absolute_percentage_error(y_test, y_pred)))